In [14]:
import geopandas as gpd
import shapely
from shapely.geometry import box, LineString, Point,MultiPoint
from shapely.ops import nearest_points
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import networkx as nx
import re
import osmnx as ox
import itertools
#from geopy import distance
from shapely import wkt
from math import radians, degrees, sin, cos, asin, acos, sqrt, floor
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import shutil

pd.options.mode.chained_assignment = None

In [20]:
folder = os.getcwd()
filename = folder + '/OutputData/graph_RAWCOLS_tnc_z_pt_bs_sc.csv'
df = pd.read_csv(filename)

,source,target,mode_type,risk_idx,discomfort,interval0_COST,interval0_avg_TT_min,interval1_avg_TT_min,interval2_avg_TT_min,interval3_avg_TT_min,...,interval3_COST,interval4_COST,interval5_COST,interval6_COST,interval7_COST,interval8_COST,interval9_COST,interval10_COST,interval11_COST,interval12_COST
0,bs49201,bs1085,bs,NaN,NaN,2.926375,NaN,NaN,NaN,NaN,...,2.926375,2.926375,2.926375,2.926375,2.926375,2.926375,2.926375,2.926375,2.926375,2.926375
1,bs49201,bs1037,bs,NaN,NaN,9.080445,NaN,NaN,NaN,NaN,...,9.080445,9.080445,9.080445,9.080445,9.080445,9.080445,9.080445,9.080445,9.080445,9.080445
2,bs49201,bs1070,bs,NaN,NaN,7.574537,NaN,NaN,NaN,NaN,...,7.574537,7.574537,7.574537,7.574537,7.574537,7.574537,7.574537,7.574537,7.574537,7.574537
3,bs49201,bs1069,bs,NaN,NaN,7.995084,NaN,NaN,NaN,NaN,...,7.995084,7.995084,7.995084,7.995084,7.995084,7.995084,7.995084,7.995084,7.995084,7.995084
4,bs49201,bs49251,bs,NaN,NaN,8.086649,NaN,NaN,NaN,NaN,...,8.086649,8.086649,8.086649,8.086649,8.086649,8.086649,8.086649,8.086649,8.086649,8.086649


In [32]:
walk_speed = 1.4 # m/s
#park_pphour = 2.50   # $; can be tuned       actually, going to the use the parking rate published with data
drive_speed = 5  # mph, speed as you go to park
num_park_hours = 4   # can be tuned
walk_speed = 1.3  # m/s
scoot_speed = 2.5  # m/s
scoot_ppmin = 0.39  # $/min
scoot_fix_price = 1  # $
bike_speed = 15 * 1000 / 3600  # m/s
bs_price = 20/300  # $/min
PT_price = 2.75  # $/ride
TNC_wait_time = 7 # min
TNC_fix_price = 2.55 + 3  # $  fixed cost + tax/fee (estimate tax+fee = 3, for simplicity)
TNC_op_ppmile = 1.75  # $/mile
TNC_op_ppmin =  1  #0.35  # $/min
pb_price = 0  # $
pv_ppmile = 0.20  # $/mile
zip_ppmin = 11/60   # $/min; still need to incorporate fixed monthly cost
zip_fixed_pmonth = 9
est_num_ziptrips = 4

# conversion factors
meters_in_mile = 1609
miles_in_km = 0.621371

# time-dependency: Consider a period of length 120 minutes (i.e. 7-9am), where each interval is 10 mintues long
interval_spacing = 10 # minutes
len_period = 120  # minutes
num_intervals = int(120/interval_spacing)+1

# beta parameters
b_price, b_TT = 1, 40/60  #21.59/60    # b_risk should be tuned
b_risk = b_TT
# for median hourly wage estimate for PA: https://www.bls.gov/oes/current/oes_pa.htm
b_disc = b_TT  # b/c defined w.r.t. travel time
b_rel = 0.5 * b_TT  # b/c defined w.r.t. travel time

# First, for every edge in the G_bs network, add a "discomfort" cost associated with biking
# For simplicity, call it [some_scalar] * avg TT. So we will think of it in terms of TT minutes
# Move this scalar to the top of the code in the params 
discomf_weight = 1/10  # TO BE TUNED!
reliability_weight_active_mode = 1
reliability_weight_nonactive_mode = 1.5

active_modes = ['w','pb','bs','sc']
nonactive_mode = ['pv','pt','t','z']  
board_alight = ['board', 'alight']  # these are special case edges

active_modes = ['w','pb','bs','sc']
nonactive_mode = ['pv','pt','t','z']  # where to put board and alight? i think they are special cases

In [33]:
folder = os.getcwd()
filename = folder + '/OutputData/graph_RAWCOLS_tnc_z_pt_bs_sc.csv'
df = pd.read_csv(filename)

price_cols = ['interval'+str(i)+'_price' for i in range(num_intervals)]
TT_cols = ['interval'+str(i)+'_avg_TT_min' for i in range(num_intervals)]
rel_cols = ['interval'+str(i)+'_reliability' for i in range(num_intervals)]
risk_cols = ['interval'+str(i)+'_risk' for i in range(num_intervals)]
cols_keep = (['source','target','mode_type', 'risk_idx', 'discomfort','interval0_COST'] 
             + TT_cols + price_cols + rel_cols + risk_cols)
df = df[cols_keep]
# add discomfort for active modes
mask = (df['mode_type'].isin(active_modes)) & (df['discomfort'].isna())
# one interval's TT is representative of all intervals b/c active modes are not td
df.loc[mask, 'discomfort'] = df.loc[mask, 'interval0_avg_TT_min'] * discomf_weight
df.loc[df['mode_type'].isin(nonactive_mode + board_alight), 'discomfort'] = 0  # for non-active modes
# add total cost as a function of the five other attributes.
# bikeshare is already calculated b/c precomputed. so do this for all other modes
bs_mask = df['mode_type'] == 'bs'

for i in range(num_intervals):
    # exception for mode_type == 'bs'
    df.loc[~bs_mask,'interval'+str(i)+'_COST'] = (b_TT * df.loc[~bs_mask,'interval'+str(i)+'_avg_TT_min'] +
                                                    b_rel * df.loc[~bs_mask,'interval'+str(i)+'_reliability'] +
                                                    b_price * df.loc[~bs_mask,'interval'+str(i)+'_price'] +
                                                    b_risk * df.loc[~bs_mask,'interval'+str(i)+'_risk'])

# bikeshare is precomputed but only for a single cost (interval0_COST). add this cost for intervals 1-num_intervals
# so that the entire table is full
for i in range(1,num_intervals):
    df.loc[bs_mask, 'interval'+str(i)+'_COST'] = df.loc[bs_mask, 'interval0_COST']
    
# only keep cost columns
cost_cols = ['interval'+str(i)+'_COST' for i in range(num_intervals)]
cols_keep = ['source','target','mode_type'] + cost_cols
df_G_final = df[cols_keep]

In [34]:
filename_1 = 'graph_FINAL_tnc_z_pt_bs_sc.csv_Btt=40'
df_G_final.to_csv(folder + '/OutputData/' + filename_1, index=False)

In [37]:
# zip the file before uploading to github
shutil.make_archive(folder + '/OutputData', 'zip', folder + '/OutputData')

'/Users/lindsaygraff/Documents/Multimodal Transit Research/Code/OutputData.zip'